#### Sociology 128D: Mining Culture Through Text Data: Introduction to Social Data Science – Summer '22

# Notebook 8: Modeling Ratings and Sentiment using Python

In this notebook, we are going to explore a range of tools for statistical analysis in Python. To do this, we are going to use the text of Yelp reviews as well as various metadata. We will conclude with regression tables that can be displayed in the notebook, copied to a word processor, or rendered in HTML or LaTeX. In the optional exercises, you will adapt the code provided in the notebook to answer your own research question.

## Setup

You will likely need to install `contractions`,  `num2words`, `pingouin`, `stargazer`, `statsmodels`, `unidecode`, and `vaderSentiment`. You can install most of these using `conda` (if using Anaconda), but you will need to install `contractions` and `stargazer` using `pip` (see below).

`conda install -c conda-forge pingouin num2words statsmodels unidecode vadersentiment`


`pip install contractions` <br>
`pip install stargazer`


In [ ]:
import contractions
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pingouin
import re
import seaborn as sns
import spacy
import statsmodels.formula.api as smf

from collections import defaultdict, Counter
from num2words import num2words
from pingouin import cronbach_alpha
from scipy.stats import pearsonr, spearmanr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.model_selection import train_test_split
from spacy.lang.en.stop_words import STOP_WORDS as spacy_stopwords
from stargazer.stargazer import Stargazer
from statsmodels.stats.weightstats import ttest_ind
from unidecode import unidecode
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
np.random.seed(2423)

## I. Data

For this notebook, we are going to use the Yelp Open Dataset, which you can find [here](https://www.yelp.com/dataset). You'll have to click 'Download Dataset', agree to the terms, and click 'Download JSON'. It's a large download: ~5GB compressed and ~11GB once you've uncompressed it. The dataset has 8,635,403 reviews of businesses including text, a rating out of five stars, and various other information. The main file, <tt>yelp_academic_dataset_review.json</tt>, is quite large, and we are going to take a random sample of it.

In [ ]:
os.listdir("data/yelp_dataset/")

First, let's confirm the number of reviews. We'll iterate through the file line by line, counting the lines by incrementing <tt>num_reviews</tt>. Then we'll use `np.random.choice` to identify the *indices* of random elements from an array of the same length. Then we'll loop back through the lines of the main file and keep the lines whose index is a match for our sample. This approach allows us to avoid loading the full dataset into memory.

In [ ]:
%%time

num_reviews = 0

with open("data/yelp_dataset/yelp_academic_dataset_review.json", "r", encoding="utf-8") as reader:
    for line in reader:
        num_reviews += 1

In [ ]:
num_reviews

Using `np.arange` with <tt>num_reviews</tt> gives us an array of the same length as <tt>num_reviews</tt> from 0 to <tt>num_reviews</tt>-1. This is more or less doing the same thing as the more familiar `range` function, but it returns a NumPy array. You can use either, but `np.arange` is a good tool to have if you start working with NumPy more.

`np.random.choice` will take a sample of the values from the array of size <tt>size</tt> without replacement.

In [ ]:
sample_indices = np.random.choice(np.arange(num_reviews), size=10000, replace=False)
len(sample_indices)

Now, we will loop through the lines of the main file using `enumerate` to count as we go. For the first line, <tt>i</tt> will be 0. For the last line, <tt>i</tt> will be 8635402. For each line, if the corresponding value of <tt>i</tt> is in <tt>sample_indices</tt>, we will append the line to the list we've called <tt>sample</tt>.

In [ ]:
%%time

sample = []

with open("data/yelp_dataset/yelp_academic_dataset_review.json", "r", encoding="utf-8") as reader:
    for i, line in enumerate(reader):
        if i in sample_indices:
            sample.append(line.strip())

JSON is a bit like a dictionary. If we use `json.loads`, we can turn each line (a string) into a dictionary.

In [ ]:
sample = [json.loads(s) for s in sample]

In [ ]:
sample[0]

In [ ]:
sample[0]["stars"]

In [ ]:
sample[0]["text"]

<tt>sample</tt> is a list of 10,000 dictionaries, each corresponding to one review. However, each fo these dictionaries has the same keys. We can use `pd.Dataframe` directly on this list to create a dataframe, and it will use the keys of the dictionaries as the columns.

In [ ]:
df = pd.DataFrame(sample)

In [ ]:
df.head()

In [ ]:
df.date.min()

In [ ]:
df.date.max()

Some quick preprocessing...

In [ ]:
def fix_ordinal_nums(word: str) -> str:
    ord_num_reg = r"\d+[(st)(nd)(rd)(th)]"
    try:
        if any(re.findall(ord_num_reg, word)):
            word = re.sub("[(st)(nd)(rd)(th)]", "", word)
            word = num2words(word, lang="en", to="ordinal")
            
        return word
    
    except:
        
        return word


def preprocess_post(post: str) -> str:
    """
    Tokenize, lemmatize, remove stop words, 
    remove non-alphabetic characters.
    """
    post = unidecode(str(post))
    post = contractions.fix(post)
    post = [word.lemma_ for word in nlp(post) if (word.text not in spacy_stopwords) & (len(word.text) > 1)]
    post = " ".join([fix_ordinal_nums(word) for word in post])
    post = re.sub("[^a-z]", " ", post.lower())
    
    return re.sub("\s+", " ", post).strip()
    

nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

In [ ]:
%time df["preprocessed"] = df.text.apply(preprocess_post)

We aren't going to use the <tt>user_id</tt> column, and <tt>review_id</tt> is also an eyesore when we try to display the dataframe. Let's drop those columns and save our work.

In [ ]:
df.drop(labels=["review_id", "user_id"], axis=1, inplace=True)
df.to_json("yelp_sample.json")

In [ ]:
df = pd.read_json("yelp_sample.json")

In [ ]:
df.head()

#### Business Metadata

Now we're going to fetch some info about the businesses themselves. First, we'll create a variable storing all of the unique business IDs in our sample. Next, we'll loop through the <tt>yelp_academic_dataset_business.json</tt> file and get info about the businesses whose IDs are in our sample.

The <tt>categories</tt> and <tt>attributes</tt> fields could be useful. They provide a lot of information about different properties of the businesses, from accessibility to ambience to type of business. <tt>categories</tt> is provided as a single string, so we're going to split that into individual categories. <tt>attributes</tt> has fewer total values, but the results are dictionaries. Some are True/False (e.g., 'Outdoor Seating') and some are more varied values (e.g., 'hours'). One of the attribute dictionaries, 'Ambience', has key/value pairs indicating whether a business is touristy, 'hipster', romantic, divey, intimate, trendy, upscale, classy, and/or casual (each True/False). We'll treat each of those as a separate attribute because we're going to use them in our analyses below. If you are interested in some of the other attributes, you may need to modify the code below to handle them how 'Ambience' is handled.

In [ ]:
businesses_in_sample = df.business_id.unique()

In [ ]:
len(businesses_in_sample)

Let's take a look at the <tt>categories</tt> and <tt>attributes</tt> fields. The code below prints the results for the first review.

In [ ]:
with open("data/yelp_dataset/yelp_academic_dataset_business.json", "r", encoding="utf-8") as reader:
    for line in reader:
        line = json.loads(line.strip())
        print(line["categories"])
        print()
        print(line["attributes"])
        break

In [ ]:
%%time

biz_categories_dict = defaultdict(lambda: {})
biz_attributes_dict = defaultdict(lambda: {})

with open("data/yelp_dataset/yelp_academic_dataset_business.json", "r", encoding="utf-8") as reader:
    for line in reader:
        line = json.loads(line.strip())
        biz_id = line["business_id"]
        if biz_id in businesses_in_sample:
            categories = line["categories"]
            attributes = line["attributes"]
            
            # check if "categories" is empty/None
            if categories:
                categories = categories.lower().split(",")
                categories = [cat.strip() for cat in categories]
                biz_categories_dict[biz_id] = set(categories)
            
            # check if "attributes" is empty/None
            if attributes:
                for key, value in attributes.items():
                    if key == "Ambience":
                        amb_categories = eval(value)
                        if type(amb_categories) == dict:
                            for cat, cat_val in amb_categories.items():
                                biz_attributes_dict[cat][biz_id] = cat_val
                    else:
                        biz_attributes_dict[key][biz_id] = value

Let's put all of the categories from <tt>categories</tt> together, count their frequencies (using `Counter`), and take a look at the most frequent.

In [ ]:
all_categories = []
for cats in biz_categories_dict.values():
    all_categories += cats

In [ ]:
len(set(all_categories))

In [ ]:
c = Counter(all_categories)
c = sorted(list(c.items()), key=lambda x: x[1], reverse=True)

In [ ]:
c[:200]

We can use <tt>biz_categories_dict</tt> and each review's <tt>business_id</tt> field to check whether each category was used to describe the business.

We can do the same for <tt>attributes</tt>.

In [ ]:
for attribute in biz_attributes_dict.keys():
    print(attribute, len(biz_attributes_dict[attribute]))

Let's add a few variables (columns) to our dataframe reflecting different properties of the businesses. Note that with <tt>RestaurantsPriceRange2</tt>, we have to change the datatype from `str` to `float`.

In [ ]:
df["RestaurantsPriceRange2"] = df.business_id.apply(lambda x: biz_attributes_dict["RestaurantsPriceRange2"].get(x, np.nan))
df.loc[df["RestaurantsPriceRange2"] == "None", "RestaurantsPriceRange2"] = np.nan
df = df.astype({"RestaurantsPriceRange2": "float"})

In [ ]:
df.RestaurantsPriceRange2.unique()

In [ ]:
df["BYOBCorkage"] = df.business_id.apply(lambda x: biz_attributes_dict["BYOBCorkage"].get(x, np.nan))
df.BYOBCorkage.unique()

In [ ]:
d = {"'no'": 0, "'yes_free'": 1, "'yes_corkage'": 1, 'None': np.nan, "u'no'": 0}
df.BYOBCorkage = df.BYOBCorkage.apply(lambda x: d.get(x, np.nan))

In [ ]:
Counter(df.BYOBCorkage.dropna())

In [ ]:
# From 'Ambience' in the raw data
df["classy"] = df.business_id.apply(lambda x: biz_attributes_dict["classy"].get(x, np.nan))

In [ ]:
df.head()

In [ ]:
df = df[df.business_id.isin(biz_categories_dict)]

In [ ]:
df.shape

In [ ]:
df["karaoke"] = df.business_id.apply(lambda x: "karaoke" in biz_categories_dict[x])
df["breweries"] = df.business_id.apply(lambda x: "breweries" in biz_categories_dict[x])
df["restaurant"] = df.business_id.apply(lambda x: "restaurant" in biz_categories_dict[x])
df["coffee"] = df.business_id.apply(lambda x: "coffee" in biz_categories_dict[x])
df["breakfast"] = df.business_id.apply(lambda x: "breakfast & brunch" in biz_categories_dict[x])

## II. Sentiment with VADER

To expand our toolkit a bit more in general and increase the range of variables we can explore in this notebook in particular, we are going to use VADER to analyze sentiment. You can check out the original paper here: [VADER: A Parsimonious Rule-based Model for  Sentiment Analysis of Social Media Text](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf). You can read more about it on the GitHub page [here](https://github.com/cjhutto/vaderSentiment).

VADER provides positive, neutral, and negative sentiment scores as well as a 'compound' score. According to the [GitHub page](https://github.com/cjhutto/vaderSentiment):
> The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

We'll focus on the compound score in our analyses. Let's print a couple of example reviews, the scores from VADER, and the number of stars the reviewer assigned.

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
for idx, row in df.sample(2).iterrows():
    print(f"Review {idx}\n--")
    print(row.text)
    vs = analyzer.polarity_scores(row.text)
    print(vs)
    print(f"Stars: {row.stars}")
    print()

...and let's go ahead and compute the scores for each review in our sample.

In [ ]:
%time df[["neg", "neu", "pos", "compound"]] = df.preprocessed.apply(lambda x: pd.Series(analyzer.polarity_scores(x)))

We'll also a variable for the day of the week that could be useful for focusing on reviews of particular types of events (e.g., dinner on Friday vs. Sunday brunch).

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df["day_name"] = df["date"].dt.day_name()

In [ ]:
df.head()

In [ ]:
df.to_json("yelp_sample.json")

## III. Descriptive Statistics

Pandas dataframes have a `describe` method that provides basic [descriptive statistics](https://en.wikipedia.org/wiki/Descriptive_statistics). You can also compute them directly (e.g., by calling `.mean()` or `.median()`) or by using a library like NumPy directly.

In [ ]:
df[["stars", "neg", "neu", "pos", "compound"]].describe()

In [ ]:
print(f"Mean: {np.mean(df.stars):.2f}")
print(f"Mean: {np.median(df.stars):.2f}")
print(f"Min: {np.min(df.stars):.2f}")
print(f"Max: {np.max(df.stars):.2f}")
print(f"Standard deviation (using n): {np.std(df.stars):.5f}") # default: formula for population standard deviation
print(f"Standard deviation (using n-1): {np.std(df.stars, ddof=1):.5f}") # use n-1 for sample standard deviation

## IV. Inferential Statistics

Inferential statistics is all about using what we know about a sample—including variability in the data, which is a source of uncertainty—to make inferences about the population from which a sample was drawn.

[Benjamin et al. (2018)](https://www.nature.com/articles/s41562-017-0189-z?source=post_page---------------------------) describe the typical approach to inferential statistics:

> In testing a point null hypothesis H<sub>0</sub> against an alternative hypothesis H<sub>1</sub> based on data x<sub>obs</sub>, the P value is defined as the probability, calculated under the null hypothesis, that a test statistic is as extreme or more extreme than its observed value. The null hypothesis is typically rejected — and the finding is declared statistically significant — if the P value falls below the (current) [type I error](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors) threshold α = 0.05.

Put differently, a p-value is our attempt to quantify the likelihood that we would observe the data we actually observe (including the test statistic derived from the data) if the null hypothesis were true. A low p-value suggests the likelihood of observing the data should be low *if the null hypothesis is correct*. We typically use a threshold like 0.05 as the cutoff for declaring statistical significance. If the p-value is >= 0.05, we *retain* the null hypothesis; if the p-value is < 0.05, we *reject* the null hypothesis.

We also refer to the 95% confience level (1 - 0.05) when α = 0.05. This is the conventional threshold, but Benjamin et al. (2018) argue against this for various reasons. [Consider this XKCD comic](https://xkcd.com/882/), for example.

The null hypothesis (H<sub>0</sub>) and the alternative hypothesis (usually referred to as H<sub>1</sub> or H<sub>A</sub>) will depend on the specific test.

#### Correlation

Correlation refers to the strength and direction of the (potential) relationship between two variables. Usually, we assume this relationship is pretty much linear. A positive correlation coefficient indicates that when one variable increases, the other does, too. A negative correlation implies that as one variable increases, the other instead decreases.

Correlation is sometimes considered to be part of descriptive statistics because correlation coefficients *describe* the strength of a relationship between two variabies. However, when we calculate a correlation coefficient, we also typically performance a hypothesis test, which is part of inferential statistics.

If we calculate the correlation coefficient between two continuous variables, our null hypothesis is that the correlation is 0 (i.e., that the variables are uncorrelated). A significant correlation coefficient suggests the variables are correlated at some confidence level (determined by the p-value).

You can calculate correlation coefficients directly using the [`.corr()` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) with your dataframe. By default, this will produce a correlation matrix using Pearson's *r*. You can add the "spearman" argument for Spearman's rho or "kendall" for the Kendall rank correlation coefficient.

In [ ]:
df[["stars", "neg", "neu", "pos", "compound", "RestaurantsPriceRange2"]].corr()

In [ ]:
df[["stars", "neg", "neu", "pos", "compound", "RestaurantsPriceRange2"]].corr("spearman")

We can also save the whole correlation matrix as a variable (here, <tt>correlation_matrix</tt>) and plot it using Seaborn's `heatmap` function.

In [ ]:
correlation_matrix = df[["stars", "neg", "neu", "pos", "compound"]].corr()
sns.heatmap(correlation_matrix, cmap="coolwarm_r", annot=True, square=True)
plt.show()

The `pingouin` library adds features like the [`rcorr` method](https://pingouin-stats.org/generated/pingouin.rcorr.html), which display correlation coefficients in the lower triangle and significance levels (represented by asterisks) in the upper triangle.

In [ ]:
df[["stars", "compound", "RestaurantsPriceRange2"]].rcorr("spearman")

`scipy.stats` makes it easy to access the correlation coefficient and p-value for specific pairs of variables.

In [ ]:
pearsonr(df.stars, df.compound)

In [ ]:
spearmanr(df.stars, df.compound)

We can access the individual pieces of information (the correlation coefficient *r* and the p-value) by assigning the results to two variables (<tt>r</tt> and <tt>pvalue</tt>) separated by a comma.

In [ ]:
r, pvalue = pearsonr(df.stars, df.compound)

In [ ]:
r

**Note: The p-value is being rounded down to zero. It's not precisely zero. We will never be that certain if we are only looking at a sample.**

In [ ]:
pvalue

#### Comparing group means with *t*-tests

To test for differences in the means between two groups, we'll use the `statsmodels` implementation of independent samples *t*-tests, `ttest_ind`. Specifically, we are going to test whether restaurants categorized as breweries and restaurants categorized as having karaoke have different ratings on average.

First, we want to make sure that our groups do not overlap (i.e., that no restaurant in our sample is a brewery that does karaoke). The code below checks whether there are any reviews for businesses that are describes as breweries and as having karaoke. There should be none in the sample.

Next we'll compare the mean star-rating for each of these groups and then conduct a *t*-test. `ttest_ind` returns the *t*-statistic, the *p*-value, and another piece of information we won't get into called the degrees of freedom. In this sample, there is not a significant difference in the mean star-rating of breweries and businesses offering karaoke.

In [ ]:
df[(df.karaoke==True) & (df.breweries==True)].shape

In [ ]:
print(df[df.karaoke==True].stars.mean())
print(df[df.breweries==True].stars.mean())

On average, breweries get an extra 0.25 stars, but the p-value is greater than 0.05, so we *retain* (or *fail to reject*) the null hypothesis. In other words, we would conclude there is no evidence of a difference between these groups.

In [ ]:
print(df[df.breweries==True].stars.mean()-df[df.karaoke==True].stars.mean())

In [ ]:
t, pvalue, _ = ttest_ind(df[df.karaoke==True].stars, df[df.breweries==True].stars, usevar="unequal")
print(f"t = {t:.2f}, p = {pvalue:.2f}")

How about restaurants that are described as classy or not?

In [ ]:
print(df[df.classy==True].stars.mean())
print(df[df.classy==False].stars.mean())

In [ ]:
print(df[df.classy==True].stars.mean()-df[df.classy==False].stars.mean())

In [ ]:
t, pvalue, _ = ttest_ind(df[df.classy==True].stars, df[df.classy==False].stars, usevar="unequal")
print(f"t = {t:.2f}, p = {pvalue:.2f}")

On average, restaurants described as classy receive an extra 0.27 stars. The p-value has been rounded down to zero, and isn't actually zero; but we can conclude that the group means are significantly different at the p < 0.01 level (or 99% confidence level).

#### Linear Regression with OLS

Now we will turn to linear regression using ordinary least squares (OLS). OLS tests the strength and significance of the relationship between the dependent variable ('outcome' or 'response' variable) and each independent variable ('predictor' or 'explanatory variable') in a way that can seem more flexible and interpretable than the previous tests. Whereas correlation coefficients are 'unit-free', a regression coefficient is interpreted as the unit change in the y variable for a unit change in the x variable. The intercept (or 'constant') in the model is the conditional mean of the dependent variable.

We are going to use the [`statsmodels` library](https://www.statsmodels.org/stable/index.html) to train our regression models and the [`stargazer` library](https://github.com/mwburke/stargazer) to format regression tables. <tt>compound</tt> is a continuous variable ranging between -1 and 1. <tt>RestaurantsPriceRange2</tt> is an ordinal variable ranging from 1 to 4, though we can treat it as a continuous variable. <tt>breakfast</tt> and <tt>classy</tt> are stored as `bools` (True/False), and `statsmodels` will implicitly treat them as categorical variables where `False` == 0 and `True` == 1. This means that they will be treated as dummy variables (another name for binary or dichotomous variables), and the coefficient for each variable indicates the change in the outcome when the variable is `True` (or 1), as opposed to `False` (or 0).

More specifically, we are going to use `statsmodels.formula.api`, which allows us to write our regression equations using R-like syntax: e.g., `y ~ x1 + x2`. See [here](https://www.statsmodels.org/devel/example_formulas.html).

In [ ]:
# We are going to create a subset of the dataframe without missing data for the variables in our models
tmp = df[["compound", "RestaurantsPriceRange2", "breakfast", "classy"]].dropna()

formula = "compound ~ RestaurantsPriceRange2 + breakfast"
ols1 = smf.ols(formula, data=tmp).fit()

formula = "compound ~ RestaurantsPriceRange2 + breakfast + classy"
ols2 = smf.ols(formula, data=tmp).fit()

In [ ]:
reg_table = Stargazer([ols1, ols2])
reg_table

`stargazer` offers numerous methods for editing the appearance of the table, for example the order the independent variables appear in, the names of the independent variables, and the title of the table.

In [ ]:
reg_table = Stargazer([ols1, ols2])
reg_table.covariate_order(["RestaurantsPriceRange2", "breakfast[T.True]", "classy[T.True]", "Intercept"])
reg_table.rename_covariates({"RestaurantsPriceRange2": "Price", "breakfast[T.True]": "Breakfast", "classy[T.True]": "Classy"})
reg_table.title("Compound Sentiment Regressed on Price, Ambience, and Breakfast")
reg_table.show_model_numbers(False)
reg_table.significance_levels([0.05, 0.01, 0.001])
reg_table.custom_columns(["Model 1", "Model 2"], [1, 1])

In [ ]:
reg_table

**Interpretation:**
For each tier increase in the measure of the restaurant's price range, we would expect the <tt>compound</tt> sentiment score of a review to increase by 0.088 on average according to Model 1 or by 0.062 on average according to Model 2, net of the other variables in each model.

Serving breakfast is associated with an increase in the <tt>compound</tt> sentiment score of approximately 0.039 (Model 1) or 0.031 (Model 2), net of the other variables in each model.

Businesses characterized as having a "classy" ambience score 0.073 higher on the <tt>compound</tt> score of sentiment on average, net of price and whether or not the business serves breakfast.

Based on a comparison of the coefficients in these two models, it appears that controlling for whether a restaurant is categorized as <tt>classy</tt> accounts for some of the association between the outcome and the other independent variables. For example, the coefficient on price decreases from 0.088 to 0.062 when we control for <tt>classy</tt>. This could be interpreted in different ways.

For example, classiness may be associated both with higher prices and with factors that lead to more positive reviews (i.e., higher <tt>compound</tt> scores). This would be an example of confounding. Alternatively, it may be that higher prices make a restaurant seem more classy, which in turn leads to more positive reviews. This would be an example of partial mediation, and there are more formal ways to test for this. The larger point is that this is observational data and as much as we may like to offer causal interpretations, we can't be certain. Our job when using tools like linear regression is to come up with the best hypothesis we can, motivated by the best theory, and then use our models (and the results of significance tests) as components of an argument.

In OLS, each coefficient is being tested using the null hypothesis (H<sub>0</sub>) that the coefficient is equal to zero. A significant coefficient on any of the independent variables indicates we can reject the null hypothesis and conclude that the two variables are related in some way. The coefficient quantifies the strength of that relationship.

Note that both the R<sup>2</sup> and Adjusted R<sup>2</sup> increase from Model 1 to Model 2. The R<sup>2</sup> is a measure of how much variance in the outcome is explained by the independent (or explanatory) variables. This tends to increase if we add additional independent variables. The adjusted R<sup>2</sup> is penalized for each additional variable, so it is slightly lower the plain R<sup>2</sup> in Model 2.

A **crucial point** with this kind of statistical modeling is that we ultimately should not care much about the R<sup>2</sup> or Adjusted R<sup>2</sup>. If we are interested in the relationship between some variable x and another variable y, we want to include additional variables only if we expect that they confound the relationship (i.e., have an effect on both x and y) or, more generally, are a source of [omitted-variable bias](https://en.wikipedia.org/wiki/Omitted-variable_bias). If you want a deeper dive into these issues, see [this paper](https://journals.sagepub.com/doi/full/10.1177/0049124118782542), although it is a reply to a comment on another paper, so a lot of the original context won't be immediately clear.

With Stargazer, can also produce LaTeX code for the table directly:

In [ ]:
print(reg_table.render_latex())

#### Logistic Regression

Finally, we will take a quick look at an example of logistic regression using `statsmodels`. Coefficients are reported in log-odds, but interpretations are otherwise similar: A unit change in x is associated with a change in the log-odds of y of some amount.

For our dependent variable, we'll use the BYOBCorkage attribute, which we have dichotomized to no == 0 and yes == 1. What factors affect whether the restaurant will uncork your BYOB wine for you?

In [ ]:
# We are going to create a subset of the dataframe without missing data for the variables in our models
tmp = df[["BYOBCorkage", "RestaurantsPriceRange2", "breakfast", "classy"]].dropna()

formula = "BYOBCorkage ~ RestaurantsPriceRange2 + breakfast"
logit1 = smf.logit(formula, data=tmp).fit()

formula = "BYOBCorkage ~ RestaurantsPriceRange2 + breakfast + classy"
logit2 = smf.logit(formula, data=tmp).fit()

logit_table = Stargazer([logit1, logit2])
logit_table.covariate_order(["RestaurantsPriceRange2", "breakfast[T.True]", "classy[T.True]", "Intercept"])
logit_table.rename_covariates({"RestaurantsPriceRange2": "Price", "breakfast[T.True]": "Breakfast", "classy[T.True]": "Classy"})
logit_table.title("BYOBCorkage Regressed on Price, Ambience, and Breakfast")
logit_table.show_model_numbers(False)
logit_table.significance_levels([0.05, 0.01, 0.001])
logit_table.custom_columns(["Model 1", "Model 2"], [1, 1])
logit_table

**Interpretation:**
According to the first model, an increase in the price range of the business is associated with a 0.298 increase in the log-odds that the business will uncork your BYOB wine. When we control for whether the restaurant is considered classy (Model 2), we no longer observe a significant relationship between price and the outcome.

Note that the lines for R<sup>2</sup> and Adjusted R<sup>2</sup> are blank for the logistic regression table. We cannot calculate a true R<sup>2</sup> or Adjusted R<sup>2</sup> using logistic regression. There are alternatives that try to quantify how well the model fits the data, but they aren't implemented here.

## V. Supervised Machine Learning

Whereas inferential statistics is concerned with generalizing from a sample to a population by quantifying uncertainty, and we often want to know about the relationship between two specific variables net of any potential confounders, that's all (usually) beside the point in supervised machine learning. We want to train models that predict outcomes well, regardless of the substantive importance of the relationship in question. Confounding is no longer an issue–we just want to predict the outcome well and in a way that generalizes to unseen data.

One thing you may notice–and which may cause some confusion–is that tools we use for inferential statistics can be used for supervised machine learning and vice versa. The two biggest categories of machine learning are *regression* (predicting continuous outcomes, like income or ratings) and *classification* (predicting membership in a category). We used OLS for linear regression in the inferential statistics section, and we can use OLS for supervised machine learning as well. Further, we used logistic regression for inferential statistics, and we can use it for classification tasks in the context of supervised machine learning.

#### Regression

In regression tasks in a supervised machine learning framework, we want to minimize an objective function like the mean squared error or root mean squared error. [Mathematically, this is equivalent to *maximizing* the R<sup>2</sup>](https://stats.stackexchange.com/questions/250730/what-is-the-mathematical-relationship-between-r2-and-mse).

In [ ]:
docs = df.preprocessed.tolist()
y = df.compound.tolist()

X_train, X_test, y_train, y_test = train_test_split(docs, y, test_size=0.3, random_state=8675309)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

regr = LinearRegression()
regr.fit(X_train, y_train)

y_pred = regr.predict(X_test)

In [ ]:
print(f"Coefficients: \n{regr.coef_}")
print(f"Mean squared error: {mean_squared_error(y_test, y_pred)}")

#### Classification

In classification problems, two major properties of our models are *precision* and *recall*. The diagram below clarifies the difference between the two. In practice, people often default to using an [F-score](https://en.wikipedia.org/wiki/F-score), such as F<sub>1</sub>, which is the harmonic mean of precision and recall.

<img src="https://raw.githubusercontent.com/soc128d/soc128d.github.io/master/assets/images/precision_recall_wiki_walber_side_by_side.png" width=800 align="left"/> <br>

([Image source](https://en.wikipedia.org/wiki/F-score#/media/File:Precisionrecall.svg))

In [ ]:
df.compound.describe()

In [ ]:
df["compound_binary"] = df.compound.apply(lambda x: x >= df.compound.median())

In [ ]:
df.compound_binary.head()

In [ ]:
%%time

docs = df.preprocessed.tolist()
labels = df.compound_binary.tolist()

X_train, X_test, y_train, y_test = train_test_split(docs, labels, test_size=0.3, random_state=8675309)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

clf = LogisticRegression(fit_intercept=False, solver="sag", penalty="l2", max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

You can read more about [`classification_report` here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html).

## VI. Exercises

<div class="alert alert-warning">
    For the exercises in this notebook, you will use the <tt>categories</tt> and <tt>attributes</tt> data to create new variables to answer a social research question. You will then use t-tests or correlation coefficients and finally a linear or logistic regression to test your hypothesis. <br><br>
    <b>Exercise 1</b><br><br>
    Keeping in mind the variables you will create, what is your research question? You might find it helpful to pose your question in terms of the relationship between two variables. At least one of these should be a continuous variable (or a variable that can be treated as continuous). It may also be helpful to use one of the pre-existing continuous variables (e.g., star ratings or sentiment) as the outcome (dependent variable). Why did you choose these variables? Why might they be related? Are there variables that you may need to control for in order to model the relationship between your two variables? You might also consider using a subset of the data confined to a particular period of time. If needed, you can start with a sample larger than the 10,000-review sample taken at the start of the notebook. (You may also find it helpful to wait to fully finish this question until you've made sure you can create the variables you have in mind.)
</div>

*Your answer here*

<div class="alert alert-warning">
    <b>Exercise 2</b><br><br>
    2.1 Now create two variables. If you are using one of the pre-existing variables as your independent or (more likely) dependent variable, you might create the other variable and then create an additional variable to use as a control variable. The variables should fit your research question. <br><br>
    Remember, you can create variables using code like the following: </div>
    
```python
df["classy"] = df.business_id.apply(lambda x: biz_attributes_dict["classy"].get(x, np.nan))
```

```python
df["karaoke"] = df.business_id.apply(lambda x: "karaoke" in biz_categories_dict[x])
```

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">
2.2 Be sure to check the values of your new variables and recode them if needed. You can using code like <tt>df.NEW_VARIABLE.unique()</tt> to display the unique values.
</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">
    <b>Exercise 3</b><br><br>
    3.1 If your independent and dependent variables are both continuous (or can be treated as continuous, e.g. <tt>RestaurantsPriceRange2</tt>), are they correlated with one another? Use <tt>pearsonr</tt> or <tt>spearmanr</tt> from <tt>scipy.stats</tt> and report both the correlation coefficient and p-value. If only one of these variables is continuous, then select two groups from your categorical variable (e.g., the True and False groups) to compare with respect to the continuous variable. What are the group means of the continuous variable? Is one mean substantially higher? Is there a significant difference according to a t-test? Use <tt>ttest_ind</tt>. Report the group means, the difference, the results of the t-test. What do the results say about your research question? Do they fit with your expectations?</div>

In [ ]:
# YOUR CODE HERE

*Your answer here*

<div class="alert alert-warning">
    3.2 Now, assuming your dependent variable is continuous, train at least two linear regression models using OLS (and <tt>smf.ols</tt>). If your dependent variable is categorical, then make sure you are using a binary version of it (i.e., only two groups) and train at least two logistic regression models using <tt>smf.logit</tt> instead. The models should be trained using the same observations and should only differ in that the second (or third, etc.) model has one additional control variable. The addition of a control variable may reflect what you write in Exercise 1. You can train the models on the same subset of observations by using code like the code below to exclude any observations missing data for any of the variables you will use:</div>
    
```python
tmp = df[["compound", "RestaurantsPriceRange2", "breakfast", "classy"]].dropna()
```

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">
    3.2 Now use <tt>stargazer</tt> to make a regression table presenting the results of your models.
</div>

In [ ]:
# YOUR CODE HERE

<div class="alert alert-warning">
    3.3 Interpret the results. Do they fit your expectations? What do they tell us about your research question?
</div>

*Your answer here*